In [1]:
import os
import glob
import cv2
import base64
from groq import Groq

# Initialize Groq client with API key
client = Groq(
   api_key=os.environ.get("GROQ_API_KEY"),
)

ModuleNotFoundError: No module named 'cv2'

In [ ]:

n = 10  # Process every nth frame
frame_dir = "frames"
report_path = "final_report.txt"  # Path for the report file

# Create frames directory if it doesn't exist
os.makedirs(frame_dir, exist_ok=True)

# Capture from the webcam (use 0 for the default MacBook camera)
cap = cv2.VideoCapture(0)
frame_count = 0
frame_interval = 10  # Take every 10th frame

# Open the report file for writing
fall_detected = False  # Track if any fall is detected
with open(report_path, "w") as report_file:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every 10th frame
        if frame_count % frame_interval == 0:
            frame_path = os.path.join(frame_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            print(f"Extracted frame: {frame_path}")
            
            # Process the extracted frame for fall detection
            with open(frame_path, "rb") as image_file:
                image_data = image_file.read()

            # Encode the image data in base64
            image_base64 = base64.b64encode(image_data).decode('utf-8')

            # Send a concise query about fall detection
            response = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI model trained to detect falls in images. Your task is to analyze the given image and determine whether a person is falling. A fall is defined as a sudden, unintended, and uncontrolled movement to the ground or a lower level, where the person appears to have lost balance, is collapsing, or is lying in an unnatural posture. If the image shows a person in a falling state or already collapsed, respond with 'Fall Detected.' If the person is standing, walking, sitting, or engaged in any normal activity, respond with 'No Fall Detected.' Your response must be strictly one of the following options: 'Fall Detected.' or 'No Fall Detected.'"
                    },
                    {
                        "role": "user",
                        "content": f"Analyze this frame: {frame_path}. Does it depict a fall?"
                    }
                ],
                model="llama-3.2-11b-vision-preview"
            )

            # Extract prediction from the response
            prediction = response.choices[0].message.content.strip()
            
            # Write the result to the report file
            report_file.write(f"Prediction for {frame_path}: {prediction}\n")
            print(f"Prediction for {frame_path}: {prediction}")

        frame_count += 1

    cap.release()

print(f"Final report generated: {report_path}")
